In [6]:
pip install keras-image-helper

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


In [18]:
pip install --user numpy==1.25.0

In [1]:
from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np
import tflite_runtime.interpreter as tflite

In [58]:
model = keras.models.load_model('model_2024_hairstyle.keras')

In [59]:
converter = tflite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()


with open ('Hairstyle_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\alaa_\AppData\Local\Temp\tmpm75xqk2k\assets


INFO:tensorflow:Assets written to: C:\Users\alaa_\AppData\Local\Temp\tmpm75xqk2k\assets


Saved artifact at 'C:\Users\alaa_\AppData\Local\Temp\tmpm75xqk2k'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2206006902096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207471875168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2206014628960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2206014631600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2206009497024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2206202357008: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [60]:
!ls -lh

total 384M
-rw-r--r-- 1 sherif 197609 154M Nov  3 14:09 best_model_ 01_0.791.h5
drwxr-xr-x 1 sherif 197609    0 Nov 16  2024 data
-rw-r--r-- 1 sherif 197609  12K Nov  3 16:44 Deploy.ipynb
-rw-r--r-- 1 sherif 197609 169K Nov  3 14:14 Hairstyle_classification.ipynb
-rw-r--r-- 1 sherif 197609  77M Nov  3 16:46 Hairstyle_model.tflite
-rw-r--r-- 1 sherif 197609 154M Nov  3 14:18 model_2024_hairstyle.keras


In [2]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

In [3]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [4]:
img = download_image(url)

img = prepare_image(img, target_size=(200,200))

In [5]:
X = np.array(img, dtype='float32')
X  = X /255

In [6]:
X = np.array([X])
X

array([[[[0.23921569, 0.40784314, 0.08627451],
         [0.25490198, 0.43137255, 0.11372549],
         [0.24705882, 0.43137255, 0.1254902 ],
         ...,
         [0.23921569, 0.34117648, 0.        ],
         [0.23921569, 0.34901962, 0.01960784],
         [0.20784314, 0.3137255 , 0.00392157]],

        [[0.24313726, 0.4117647 , 0.10196079],
         [0.2509804 , 0.41960785, 0.10980392],
         [0.25490198, 0.41960785, 0.12156863],
         ...,
         [0.2509804 , 0.34117648, 0.        ],
         [0.24313726, 0.33333334, 0.01176471],
         [0.21568628, 0.32156864, 0.01960784]],

        [[0.26666668, 0.4392157 , 0.13725491],
         [0.2627451 , 0.42745098, 0.12941177],
         [0.27450982, 0.42352942, 0.12941177],
         ...,
         [0.2509804 , 0.34901962, 0.03137255],
         [0.23137255, 0.32156864, 0.01568628],
         [0.20392157, 0.29803923, 0.        ]],

        ...,

        [[0.10196079, 0.2       , 0.04313726],
         [0.10196079, 0.2       , 0.04313726]

In [7]:
interpreter = tflite.Interpreter('Hairstyle_model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
input_index, output_index

(0, 13)

In [8]:
interpreter.set_tensor(input_index, X)

interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [9]:
classes = [
    'Curly',
    'Straight'
]

In [10]:
preds

array([[0.8937725]], dtype=float32)